# Using PyBQL to Access Fundamental Data

Before we start, this notebook includes some advanced BQL functionality. The reader is encouraged to review our first two tutorials on PyBQL.
<br> <a href="Data - pybql Getting Started Part 1.ipynb"> Data - pybql Getting Started Part 1.ipynb </a>
<br> <a href="Data - pybql Getting Started Part 2.ipynb">Data - pybql Getting Started Part 2.ipynb </a>  

This notebook will review a few ways to use fundamental data including:
- <a href="#Grouped-Analysis">Grouped Analysis</a>
- <a href="#Combine-Data">Combine Data (both historical and forward estimates)</a>
- <a href="#Point-in-Time-Data">Point-in-Time Data</a>
- <a href="#Calendarization">Calendarization</a>
- <a href="#Fundamentals-in-Different-Currencies">Fundamentals in Different Currencies</a>
- <a href="#Fixed-Income-Screen">Fixed Income Screen</a>

Over 14,000 fundamental fields are available with BQL. The BQL service allows users to be more productive in many ways including:
- Use of a single field for reported and estimated fields (e.g., no need to use IS_EPS and BEST_EPS).
- True point-in-time addressability (use of revision date, as of date and period end date).
- Data retrieval and analytics on data with both fiscal and calendar dimensions.

---
# Grouped Analysis

# Let's start by examining how the S&P 500 Debt Ratio (Total Debt to Total Assets) has changed over time. We will use BQL to group data from all members in a given index. Data will be from 2006 to 2015.

# Our BQL string query will use:
## `let` to define variables within our query
- `znav` is used to handle null values (returns 0 if the value is not available)
- `group` is used to group securities by period end date
- `sum` is used to add values together with the same period end date
- Variables in the `let` clause must be prefaced with a hash symbol `#`

This statement allows us to group on period end date for all securities in the S&P 500. 

```
let(
    #tot_debt      = znav(bs_lt_borrow) + znav(bs_st_borrow);
    #tot_debt_agg  = sum(group(#tot_debt, by=#tot_debt().period_end_date));
    #tot_assets_agg = sum(group(bs_tot_asset, by=bs_tot_asset().period_end_date));
    #grouped_leverage = #tot_debt_agg / #tot_assets_agg;)
```

## `get` to define the data we want
Note: we defined `grouped_leverage` with the `let` clause.

```
get(#grouped_leverage)
```

## `for` to define our universe

```
for(members('SPX Index'))
```

## `with` to apply parameters to the entire query
Note: this is particularly helpful if you're requesting more than one field.

```
with(currency=USD, 
     fa_period_reference=range('2006', '2015'), 
     fa_period_type=Q, 
     fa_period_year_end=C)
```

In [1]:
import bql
from bqplot import pyplot as plt

bq = bql.Service()

multi_query = bq.execute("""
let(
    #tot_debt         = znav(bs_lt_borrow) + znav(bs_st_borrow);
    #tot_debt_agg     = sum(group(#tot_debt, by=#tot_debt().period_end_date));
    #tot_assets_agg   = sum(group(bs_tot_asset, by=bs_tot_asset().period_end_date));
    #grouped_leverage = #tot_debt_agg / #tot_assets_agg;)
get(#grouped_leverage)
for(members('SPX Index'))
with(currency=USD, 
     fa_period_reference=range('2006', '2015'), 
     fa_period_type=Q, 
     fa_period_year_end=C)
""")

# Convert the BQL response into a Pandas DataFrame.
# Use to combined_df method from BQL to merge the single item responses.
# See below for more information on single item responses.
data = bql.combined_df(multi_query)

In [2]:
data.tail()

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,ORIG_IDS,#grouped_leverage
ID,,,,,,
2011-12-31T00-00-00Z,2014-07-24,2020-05-15,2011-12-31,None,None,0.224168
2012-12-31T00-00-00Z,2015-12-19,2020-05-15,2012-12-31,None,None,0.215656
2013-12-31T00-00-00Z,2016-06-26,2020-05-15,2013-12-31,None,None,0.212375
2014-12-31T00-00-00Z,2017-09-25,2020-05-15,2014-12-31,None,None,0.216436
2015-12-31T00-00-00Z,2018-08-28,2020-05-15,2015-12-31,None,None,0.225094


In [3]:
# We will use pyplot from bqplot to visualize the data.
plt.figure(title='S&P 500 Debt Ratio')
plt.plot(data['PERIOD_END_DATE'].dropna(), data['#grouped_leverage'])
plt.axes(options={'x': {'label': 'Year', 
                                'grid_lines': 'solid'},
                  'y': {'label': 'Total Debt to Total Assets', 
                                'grid_lines': 'solid'}})
plt.show()

# In addition to BQL's `group function` we can use Pandas to arrive at similar results. It's important to note that PyBQL imports Pandas into its own namespace. This gives us the ability to easily convert a BQL response into a Pandas DataFrame. For example:

```Python
df = response[0].df() # Used for single item responses. 
```

```Python
df = bql.combined_df(response) # Used for multi item responses. 
```

`bql.combined_df()` does its best job at merging multiple DataFrames (remember that BQL will return a separate response for every item/field requested) but the user may need to use Pandas to merge multiple DataFrames together. We have an example below to help demonstrate this. 

In [4]:
import bql
from bqplot import pyplot as plt

bq = bql.Service()

multi_query = bq.execute("""
get (bs_st_borrow, bs_lt_borrow, bs_tot_asset)
for(members('SPX Index'))
with(currency=USD, 
     fa_period_reference=range('2006', '2015'), 
     fa_period_type=Q, 
     fa_period_year_end=C)
""")

data = bql.combined_df(multi_query)

In [5]:
multi_query[0].df()

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,bs_st_borrow
ID,,,,,
LYB UN Equity,NaT,2019-10-20,2006-12-31,USD,NaN
LYB UN Equity,NaT,2019-10-20,2007-12-31,USD,NaN
LYB UN Equity,NaT,2019-10-20,2008-12-31,USD,NaN
LYB UN Equity,2012-02-29,2019-10-20,2009-12-31,USD,6.679000e+09
LYB UN Equity,2013-02-12,2019-10-20,2010-12-31,USD,4.600000e+07
LYB UN Equity,2014-02-20,2019-10-20,2011-12-31,USD,5.200000e+07
LYB UN Equity,2015-02-17,2019-10-20,2012-12-31,USD,9.600000e+07
LYB UN Equity,2016-02-02,2019-10-20,2013-12-31,USD,5.900000e+07
LYB UN Equity,2017-02-17,2019-10-20,2014-12-31,USD,3.500000e+08


In [8]:
multi_query[1].df()

In [9]:
data.sample(10)

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,bs_st_borrow,bs_lt_borrow,bs_tot_asset,total_debt_out
ID,,,,,,,,
PLD UN Equity,2012-02-29,2019-08-15,2009-12-31,USD,2.116042e+09,1.096554e+09,6.841958e+09,3.212596e+09
SNPS UW Equity,2010-03-09,2019-08-15,2008-12-31,USD,0.000000e+00,0.000000e+00,2.636658e+09,0.000000e+00
V UN Equity,2017-02-02,2019-08-15,2015-12-31,USD,0.000000e+00,1.587700e+10,5.497700e+10,1.587700e+10
LNT UW Equity,2018-02-23,2019-08-15,2015-12-31,USD,4.732000e+08,3.522200e+09,1.249520e+10,3.995400e+09
AMCR UN Equity,NaT,2019-08-15,2007-12-31,USD,0.000000e+00,0.000000e+00,NaN,0.000000e+00
MAS UN Equity,2012-02-21,2019-08-15,2009-12-31,USD,3.640000e+08,3.604000e+09,9.175000e+09,3.968000e+09
HFC UN Equity,2010-02-26,2019-08-15,2009-12-31,USD,0.000000e+00,7.074580e+08,3.145939e+09,7.074580e+08
PCAR UW Equity,2015-02-26,2019-08-15,2012-12-31,USD,3.562700e+09,4.317400e+09,1.862780e+10,7.880100e+09
JKHY UW Equity,2015-06-25,2019-08-15,2014-12-31,USD,7.788800e+07,9.230000e+05,1.620971e+09,7.881100e+07


In [11]:
# Replace NaN values with zero.
data['bs_st_borrow'].fillna(value=0, inplace=True)
data['bs_lt_borrow'].fillna(value=0, inplace=True)

# Create total debt outstanding column.
data['total_debt_out'] = data['bs_st_borrow'] + data['bs_lt_borrow']

# Group and sum all values by period end date. Creates
# a single value that represents the entire index. 
grouped_df = data.groupby('PERIOD_END_DATE').sum()
grouped_df

,bs_st_borrow,bs_lt_borrow,bs_tot_asset,total_debt_out
PERIOD_END_DATE,,,,
2006-12-31,2.781058e+12,2.789472e+12,1.987862e+13,5.570530e+12
2007-12-31,3.129582e+12,3.320948e+12,2.222870e+13,6.450530e+12
2008-12-31,2.748969e+12,3.813398e+12,2.273774e+13,6.562367e+12
2009-12-31,1.895544e+12,4.028132e+12,2.397782e+13,5.923676e+12
2010-12-31,2.013388e+12,4.100352e+12,2.528179e+13,6.113739e+12
2011-12-31,1.975337e+12,3.923754e+12,2.640743e+13,5.899092e+12
2012-12-31,1.932632e+12,4.130957e+12,2.821113e+13,6.063589e+12
2013-12-31,1.925584e+12,4.283958e+12,2.928693e+13,6.209542e+12
2014-12-31,1.823576e+12,4.749388e+12,3.042962e+13,6.572964e+12


In [3]:
# Create debt ratio column for the entire index.
grouped_df['debt_ratio'] = grouped_df['total_debt_out'] / grouped_df['bs_tot_asset']

# Plot results.
plt.figure(title='S&P 500 Debt Ratio')
plt.plot(grouped_df.index.values, grouped_df['debt_ratio'])
plt.axes(options={'x': {'label': 'Year', 
                                'grid_lines': 'solid'},
                  'y': {'label': 'Total Debt to Total Assets', 
                                'grid_lines': 'solid'}})
plt.show()

---
# Combine Data

# Now we'll retrieve more than one period of data (combining both historical and forward estimates) for multiple securities.

# Note the use of a range with `fa_period_offset`

```
fa_period_offset=range('-1', '3')
```

By using `range('-1', '3')`, we are asking BQL to return data one period before and three periods after the `AS_OF_DATE`. This is important because in this example we're asking for data on periods in the future. 

In [ ]:
import bql
bq = bql.Service()

request = ("""
get(SALES_REV_TURN(fa_period_offset=range('-1', '3'), 
                   fa_period_type='A', 
                   as_of_date='2016-06-30') / 1000000000 as #Sales_in_billions) 
for(['IBM US Equity', 'AAPL US Equity'])
""")

response = bq.execute(request)
df = bql.combined_df(response)
df

Let's review values for IBM US Equity in the above DataFrame. Depending on the day you run this notebook you may have different forward sales estimates (due to broker revisions).

|                    |CURRENCY |REVISION_DATE  |AS_OF_DATE  |PERIOD_END_DATE  |#Sales_in_billions |
|--------------------|---------|---------------|------------|-----------------|---------------|
|ID                  |         |               |            |                 |               |
|IBM US Equity       |USD      |2016-02-23     |2016-06-30  |2014-12-31       |92.793000      |
|IBM US Equity       |USD      |2016-02-23     |2016-06-30  |2015-12-31       |81.741000      |       
|IBM US Equity       |USD      |2016-06-29     |2016-06-30  |2016-12-31       |79.368526      |
|IBM US Equity       |USD      |2016-06-29     |2016-06-30  |2017-12-31       |79.239667      |
|IBM US Equity       |USD      |2016-06-29     |2016-06-30  |2018-12-31       |78.876000      |

There are a few important things to take note of:
- We used a single field, `SALES_REV_TURN`, to get both historical (reported) values and forward consensus estimates.
- We can easily scale our results. In this case we return a value in billions by adding `/ 1000000000` after `SALES_REV_TURN()` but before the close of our `get` clause.
- We used `Sales_in_billions` to rename our column header. Note: similar to the `let` clause, variable names must be prefaced with a hash symbol `#`.

```python
request = ("""
get(SALES_REV_TURN(fa_period_offset=range('-1', '3'), 
                   fa_period_type='A', 
                   as_of_date='2016-06-30') / 1000000000 as #Sales_in_billions) 
for(['IBM US Equity', 'AAPL US Equity'])
""")               
```


### Overrides we used

|Parameter             |Type   |Example                     |Alias          |Description          | 
|----------------------|-------|----------------------------|---------------|---------------------|
|fa_period_offset      |string |`fa_period_offset=range('-1', '3')`|`fpo=range('-1', '3')`|Number of periods to retrieve. A range may be specified.|
|fa_period_reference   |string |`fa_period_reference=range('2002', '2015')` <br><br> `fa_period_reference='2006-09-30'`   |`fpr=range('2002', '2015')`| Reference date for the period_end_date.|
|fa_period_type        |string |`fa_period_type='A'` | `fpt='A'`        |Periodicity as a string (Annual, Semiannual and Quarterly).|       
|as_of_date            |string |`as_of_date='2016-06-30'` | `aod='2016-06-30'` |Retrieve data reported as of a specific date (helpful for point-in-time data). |

---
# Point-in-Time Data

With the use of `REVISION_DATE`, `AS_OF_DATE`, and `PERIOD_END_DATE` we are able construct a true representation of point-in-time data.

## `REVISION_DATE`
`REVISION_DATE` is the most recent date a given value was reported/revised/reiterated. It's important to note, especially with estimates, that the revision date could change often even though a given value didn't change from the previous period. Bloomberg consensus estimates are a composite of many different brokers. As such, if one of the brokers published a new report, and even if the estimate didn't change, the `REVISION_DATE` will change accordingly to reflect the new report date.

In short, the `REVISION_DATE` tells us that a given value was either reported/revised/reiterated in a new report or financial statement. The `REVISION_DATE` will be less than or equal to the `AS_OF_DATE`.

## `AS_OF_DATE`
`AS_OF_DATE` corresponds to the date when a given value was retrieved. This is particularly important when retrieving point-in-time data. With this override we're able to find specific values that were known at a specific date in the past. The `AS_OF_DATE` can either be before or after the `PERIOD_END_DATE` depending on whether we're looking at estimates or historical (reported) values. 

## `PERIOD_END_DATE`
`PERIOD_END_DATE` is the date associated with the accounting period for a given value. For most firms `PERIOD_END_DATE` are quarter end dates such as March 31, June 30, September 30 and December 31. 

## Let's take a closer look

In the following DataFrame we can see that we wanted to know what the December 31, 2014 sales value was as of June 30, 2016. We also know that that value was reported/revised/reiterated on February 23, 2016. We can always verify this by looking at both `FA <GO>` (financial analysis) and `CF <GO>` (company filings) from the Bloomberg terminal.

```python
request = ("""
get(SALES_REV_TURN(fa_period_reference='2014', 
                   fa_period_type='A', 
                   as_of_date='2016-06-30') / 1000000000 as #Sales_in_billions)
for(['IBM US Equity'])
""")
```

|                      |CURRENCY | REVISION_DATE | AS_OF_DATE | PERIOD_END_DATE | #Sales_in_billions|
|----------------------|---------|---------------|------------|-----------------|-------------------|
|ID                    |         |               |            |                 |                   |
|IBM US Equity         |USD      |2016-02-23     |2016-06-30  |2014-12-31       |92.793000          |


In the next DataFrame we can see that the December 31, 2018 estimated sales value that was known on June 30, 2016 was last reported/revised/reiterated the day before on June 29, 2016. Given the nature of broker estimates, the Bloomberg consensus will often have a revision date that changes more frequently than reported (actual) values from the financial statements.

```python
request = ("""
get(SALES_REV_TURN(fa_period_reference='2018', 
                   fa_period_type='A', 
                   as_of_date='2016-06-30') / 1000000000 as #Sales_in_billions)
for(['IBM US Equity'])
""")
```

|                      |CURRENCY | REVISION_DATE | AS_OF_DATE | PERIOD_END_DATE | #Sales_in_billions|
|----------------------|---------|---------------|------------|-----------------|-------------------|
|ID                    |         |               |            |                 |                   |
|IBM US Equity         |USD      |2016-06-29     |2016-06-30  |2018-12-31       |78.876000          |

# The following example will show how the forward sales estimate for IBM US Equity changed over time from January 2014 to December 2015. 

In [4]:
import bql
bq = bql.Service()

request = ("""
get(SALES_REV_TURN(fa_period_reference='2015-12-31', 
                   fa_period_type='A',
                   fill='prev',
                   as_of_date=range('2014-01-01', '2015-12-31')) / 1000000000 as #Sales_in_billions)
for(['IBM US Equity'])
""")

response = bq.execute(request)
df = bql.combined_df(response)
df.head(10)

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,#Sales_in_billions
ID,,,,,
IBM US Equity,2013-12-17,2014-01-01,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-02,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-03,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-04,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-05,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-06,2015-12-31,USD,103.456571
IBM US Equity,2013-12-17,2014-01-07,2015-12-31,USD,103.456571
IBM US Equity,2014-01-08,2014-01-08,2015-12-31,USD,103.554125
IBM US Equity,2014-01-08,2014-01-09,2015-12-31,USD,103.554125


# Before we move on let's find IBM's actual reported sales for December 31, 2015

In [5]:
import bql
bq = bql.Service()

request = ("""
get(SALES_REV_TURN(fa_period_reference='2015-12-31', 
                   fa_period_type='A') / 1000000000 as #Sales_in_billions)
for(['IBM US Equity'])
""")

response = bq.execute(request)
df_actual = response[0].df()
df_actual

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,#Sales_in_billions
ID,,,,,
IBM US Equity,2018-02-27,2019-08-05,2015-12-31,USD,81.741


# Using  the `as_of_date` we're able to observe how the Bloomberg consensus value changed over time.

# This visualization allows us to easily see that brokers revised their estimates to ultimately converge with the actual reported value of $81.7B.

In [6]:
from bqplot import pyplot as plt
import numpy as np

plt.figure(title='IBM - Estimated Revenue (Bloomberg Consensus)')
plt.plot(df['AS_OF_DATE'], df['#Sales_in_billions'])
plt.axes(options={'x': {'label': 'Date (Jan 2014 to Dec 2015)', 
                                'grid_lines': 'solid'},
                  'y': {'label': 'Revenue (Billions)', 
                                'grid_lines': 'solid'}})

# Add a label to highlight the actual sales value of $81B.
plt.label(['12/31/15 Actual Sales $81.7B'], 
          x=[np.datetime64(df['AS_OF_DATE'].iloc[-400])], 
          y=[df['#Sales_in_billions'].iloc[-50]], 
          colors=['yellow'],
          scales={'x': plt._get_context_scale('x'), 
                  'y': plt._get_context_scale('y')})


plt.show()

---
# Calendarization
BQL allows us to easily align values into a single calendar year (especially important for companies with irregular fiscal calendars).

Let's take a look at Microsoft Corp, MSFT US Equity, whose fiscal year ends June 30th. We'll retrieve both fiscal and calendar year end value for sales. We'll then reconcile the calendar and year end values. 

In [3]:
import bql
bq = bql.Service()

# Note that we use C (calendar) for fa_period_year_end.
# F (fiscal) is also used to retrieve fiscal year end values.
request = ("""
get(SALES_REV_TURN(fa_period_year_end='C',
                   fa_period_type='A',
                   fa_period_reference=range('2014', '2015')
                   ) / 1000000000 as #Sales_calendar_in_billions, 
    SALES_REV_TURN(fa_period_year_end='F',
                   fa_period_type='A', 
                   fa_period_reference=range('2014', '2015')
                   ) / 1000000000 as #Sales_fiscal_in_billions)
for(['MSFT US Equity'])
""")

response = bq.execute(request)
df = bql.combined_df(response)
df

,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,CURRENCY,#Sales_calendar_in_billions,#Sales_fiscal_in_billions
ID,,,,,,
MSFT US Equity,2016-07-28,2019-08-05,2014-12-31,USD,93.456,NaN
MSFT US Equity,2017-08-03,2019-08-05,2015-12-31,USD,90.072,NaN
MSFT US Equity,2016-07-28,2019-08-05,2014-06-30,USD,NaN,86.833
MSFT US Equity,2017-08-02,2019-08-05,2015-06-30,USD,NaN,93.580


# The query above returned 2014 and 2015 `SALES_REV_TURN` for both the calendar and fiscal year. 

First, it's important to note the differences in `PERIOD_END_DATE`. Calendar (year end) values end December 31st and fiscal  values (for Microsoft) end June 30th. 

Before we move on let's take note of the 2014 calendar sales value of `$93.456B`. We will use this in our reconciliation below.

We'll start by obtaining the calendar quarterly values for Microsoft Corp.

In [ ]:
import datetime
import bql

bq = bql.Service()

request = ("""
get(SALES_REV_TURN(fa_period_type='Q',
                   fa_period_reference=range('2013Q1', '2015Q4')
                   ) / 1000000000 as #Sales_in_billions)
for(['MSFT US Equity'])
""")

# Execute the query.
response = bq.execute(request)

# Convert the BQL response into a DataFrame.
df = response[0].df()

# Define our start and end dates.
start = datetime.date(2014, 1, 1)
end = datetime.date(2014, 12, 31)

# We will use the Pandas loc method to find values
# within our start and end dates. Before we do that
# we need to set our index to be the PERIOD_END_DATE.
df = df.set_index('PERIOD_END_DATE')

# The Pandas loc method allows us to use labels in the 
# index to slice the DataFrame and only return values we need.
# When slicing the start and end boundaries are included.
df.loc[start:end]

# Reconciliation
Now we'll add the quarterly values together to arrive at the same figure above, `$93.456B`.

In [ ]:
fiscal_year = df.loc[start:end].sum()
fiscal_year.values

---
# Fundamentals in Different Currencies
BQL allows us to easily retrieve fundamental values in different currencies. 

In [ ]:
import bql
bq = bql.Service()

request = bq.execute("""
get(SALES_REV_TURN(currency=EUR)  / 1000000000 as #sales_in_eur,
    SALES_REV_TURN(currency=USD) / 1000000000 as #sales_in_usd,
    BS_TOT_ASSET(currency=EUR) / 1000000000 as #assets_in_eur,
    BS_TOT_ASSET(currency=USD) / 1000000000 as #assets_in_usd
)
for(['IBM US Equity'])
""")

df = bql.combined_df(request)
df

---

# Fixed Income Screen


# BQL allows us to create a credit scoring model by screening fixed income instruments.

The following example will use a modified version of Edward Altman's Z Score to measure relative financial distress for firms in the Dow Jones Industrial Average.

In [6]:
import bql
bq = bql.Service()

request = """
let (
    #working_cap = bs_cur_asset_report / avg(bs_tot_asset(
        fa_period_offset=range(-1, 0)));
    #retained_earn = bs_retain_earn / avg(bs_tot_asset(
        fa_period_offset=range(-1, 0)));
    #ebit_roa = is_ebit / avg(bs_tot_asset(
        fa_period_offset=range(-1, 0)));
    #equity = cur_mkt_cap / avg(bs_tot_liab_eqy(
        fa_period_offset=range(-1, 0)));
    #sales = sales_rev_turn / avg(bs_tot_asset(
        fa_period_offset=range(-1, 0)));
    
    #z_score = 1.2 * #working_cap
            + 1.4 * #retained_earn
            + 3.3 * #ebit_roa
            + 0.6 * #equity
            + 1.0 * #sales;)
get(#z_score)
for(members('INDU Index'))"""


response = bq.execute(request)
df = bql.combined_df(response)

# Drop securities (with NAs) that don't have a valid Z Score.
df = df['#z_score'].dropna().to_frame()
df.head()

,#z_score
ID,
VZ UN Equity,1.649881
BA UN Equity,1.489587
CAT UN Equity,2.159546
CVX UN Equity,1.797381
KO UN Equity,2.380712


# Let's graph these results as a simple scatter plot. We will add a tooltip to allow the user to see the Ticker and Z Score for a given value.

In [7]:
from bqplot import pyplot as plt
from bqplot import Tooltip

data = df['#z_score']

plt.figure(title="Distribution of Z Scores (high value is better)")
x_data = list(data.index)
y_data = list(data.values)

# Define tooltip.
def_tt = Tooltip(fields=['x', 'y'], 
                 formats=['', '.2f'], 
                 labels=['Ticker', 'Z Score'])

# Add scatter plot with x, y data and tooltip.
plt.scatter(x=x_data, y=y_data, tooltip=def_tt)

plt.axes(options={'x': {'visible': False},
                  'y': {'label': 'Z Score'}})
plt.show()

# Now that we have a list of Z Scores, let's find bonds of companies with a Z score greater than  or equal to 3. We will use the PyBQL object model for this example.

In [8]:
# List of companies with Z score greater than 3.
high_z_score = df['#z_score'] >= 3
company_list = df[high_z_score]
company_list.head()

,#z_score
ID,
HD UN Equity,6.716414
JNJ UN Equity,3.422095
WMT UN Equity,4.290438
CSCO UW Equity,3.269009
INTC UW Equity,3.073236


In [9]:
# Convert company_list DataFrame into a list of tickers.
# Use this list in a BQL query to get a list of bonds.
companies = company_list.index.tolist()

from collections import OrderedDict
import bql
bq = bql.Service()

# Fields as an ordered Python dictionary.
# This ensures our DataFrame columns are in
# the same order as our request. 
fields_ordered_dict = OrderedDict()
fields_ordered_dict["maturity"] = bq.data.maturity()
fields_ordered_dict["coupon"] = bq.data.cpn()
fields_ordered_dict["moody_rating"] = bq.data.rtg_moody()

# Get all bonds for each ticker in the companies list.
bond_universe = bq.univ.bonds(companies)

# Define BQL request object.
request = bql.Request(bond_universe, fields_ordered_dict)

# Execute the request object.
response = bq.execute(request)

# Return a combined DataFrame.
bql.combined_df(response).head()

,maturity,CPN_TYP,MULTIPLIER,coupon,moody_rating
ID,,,,,
BJ262209 Corp,2025-05-11,None,NaN,NaN,Aa1
BJ262213 Corp,2030-05-11,None,NaN,NaN,Aa1
BJ262218 Corp,2050-05-11,None,NaN,NaN,Aa1
BJ262206 Corp,2023-05-11,None,NaN,NaN,Aa1
EJ659248 Corp,2023-05-03,None,NaN,NaN,Aa1


# Let's use some `Pandas` magic to merge our three BQL responses into a single DataFrame. First we'll look at each individual DataFrame to see what column headers we need to merge on.

In [10]:
# Create individual DataFrames and reset the index.
maturity_df = response.get('maturity').df().reset_index()
coupon_df = response.get('coupon').df().reset_index()
rating_df = response.get('moody_rating').df().reset_index()

In [11]:
# maturity DataFrame
maturity_df.head()

,ID,maturity
0,BJ262209 Corp,2025-05-11
1,BJ262213 Corp,2030-05-11
2,BJ262218 Corp,2050-05-11
3,BJ262206 Corp,2023-05-11
4,EJ659248 Corp,2023-05-03


In [12]:
# coupon DataFrame
coupon_df.head()

,ID,CPN_TYP,MULTIPLIER,coupon
0,BJ262209 Corp,FIXED,1.0,1.125
1,BJ262213 Corp,FIXED,1.0,1.650
2,BJ262218 Corp,FIXED,1.0,2.650
3,BJ262206 Corp,FIXED,1.0,0.750
4,EJ659248 Corp,FIXED,1.0,2.400


In [13]:
# Moody's rating DataFrame
rating_df.head()

,ID,moody_rating
0,BJ262209 Corp,Aa1
1,BJ262213 Corp,Aa1
2,BJ262218 Corp,Aa1
3,BJ262206 Corp,Aa1
4,EJ659248 Corp,Aa1


In [14]:
merged_df = maturity_df.merge(
            coupon_df, left_on=("ID"), 
                       right_on=("ID")).merge(
            rating_df, left_on=("ID"), right_on=("ID"))

merged_df.head()

,ID,maturity,CPN_TYP,MULTIPLIER,coupon,moody_rating
0,BJ262209 Corp,2025-05-11,FIXED,1.0,1.125,Aa1
1,BJ262213 Corp,2030-05-11,FIXED,1.0,1.650,Aa1
2,BJ262218 Corp,2050-05-11,FIXED,1.0,2.650,Aa1
3,BJ262206 Corp,2023-05-11,FIXED,1.0,0.750,Aa1
4,EJ659248 Corp,2023-05-03,FIXED,1.0,2.400,Aa1


---
To the extent possible under law, Bloomberg L.P. has waived all copyright and related or neighboring rights to this notebook, 
in accordance with the Creative Commons CC0 1.0 Public Domain Dedication (http://creativecommons.org/publicdomain/zero/1.0/).